# This is brief tutoral on install angsd-wrapper and running ngsF
    This uses jahner's bighorn data as an example
    Below lines should all be ran in the command line on pronghorn
    This is setup to be ran on pronghorn with slurm. Could be modified 

# Create new conda environment

In [ ]:
conda create -n angsdWrap
source activate angsdWrap
conda install -c anaconda gcc
conda install -c anaconda libgcc
conda install -c bioconda samtools

# Install angsd-wrapper 
    https://github.com/ANGSD-wrapper/angsd-wrapper
    
    This should be done in a src directory in your preffered working directoty
    for example: /data/gpfs/assoc/parchmanlab/tfaske/src

In [ ]:
cd /PATH/src

source activate angsdWrap
git clone https://github.com/mojaveazure/angsd-wrapper.git
cd angsd-wrapper/
./angsd-wrapper setup dependencies
source ~/.bash_profile

if you get an gsl error....

In [ ]:
#try to install with conda
conda install -c conda-forge gsl

if this does not work, add gsl install from source in parchmanlab association to ~/.bashrc

In [ ]:
# add gsl
export PATH="/data/gpfs/assoc/parchmanlab/src/gsl/bin:$PATH"

# add gsl LD lib path
export LD_LIBRARY_PATH="/data/gpfs/assoc/parchmanlab/src/gsl/lib:$LD_LIBRARY_PATH"
export CPATH=/data/gpfs/assoc/parchmanlab/src/gsl/include/
export LIBRARY_PATH=/data/gpfs/assoc/parchmanlab/src/gsl/lib/

After gsl is added to conda or ~/.bashrc, logout and log back in and repeat installation steps

# Get necessary example files and additional shell scripts

In [ ]:
cd PATH/src/angsd-wrapper/

source activate angsdWrap
./angsd-wrapper setup data

Get regions extractor shell script, explained later or on website

In [ ]:
cd PATH/src/angsd-wrapper/

wget https://gist.githubusercontent.com/mojaveazure/d115bb25eeff3b2df9f9/raw/87d84bcd1a8e9f705d4b3b37639a50f0fd3e8e46/regionsExtracter.sh

chmod 755 regionsExtracter.sh

# Running ngsF
    tutorial through angsd-wrapper: https://github.com/mojaveazure/angsd-wrapper/wiki/Inbreeding-Coefficients
    OG: https://github.com/fgvieira/ngsF
    
    Below code can be ran using jupyter notebooks
    
    Main directory should have 2 directories: 
    bam_files -- containing individual bamfiles
    assembly -- contains assembly
    
    The only thing a few things need to be changed in below scripts:
    root_dir path, angsdWrap_dir, assembly files name, cpus, and various file names 

In [80]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [81]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd

In [82]:
root_dir = '/data/gpfs/home/tfaske/g/jahner/bighorn'

In [83]:
bam_dir = os.path.join(root_dir,'bam_files')
assembly_dir = os.path.join(root_dir,'assembly')
ngsF_dir = os.path.join(root_dir,'ngsF')

In [84]:
cd $root_dir

/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn


In [17]:
!mkdir ngsF

In [85]:
!find $bam_dir -name '*.bam' > ngsF/bam_list.txt

In [86]:
bam_list = os.path.join(ngsF_dir,'bam_list.txt')

# Both assembly and bam files need to be indexed

    reindex assembly with samtools (creates .fai file )

In [87]:
assembly = os.path.join(assembly_dir,'aries_genome.fa')
assembly

'/data/gpfs/home/tfaske/g/jahner/bighorn/assembly/aries_genome.fa'

In [88]:
!samtools faidx $assembly

    reindex bam files with samtools as well (creates .bai file )

In [89]:
bam_files = !find $bam_dir -name '*sorted.bam'
len(bam_files),bam_files[0]

(156,
 '/data/gpfs/home/tfaske/g/jahner/bighorn/bam_files/aln_DB_183_1401.sorted.bam')

In [90]:
for bam in bam_files:
    !samtools index $bam

In [91]:
#checks and makes sure all were done
bam_index = !find $bam_dir -name '*.bam'
assert bam_files == bam_index

# Designates contigs / chromosomes / scaffolds of interest for estimation
    
    If you have a genome, maybe restrict analyses to a set number of largest scaffolds or only chromosomes
    Example file should look like
    scaffold_1:
    scaffold_2:
    scaffold_3: 
    etc……
    
    For denovo, pick an artibutary number and run regionsExtractor.sh or run on all contigs. 
    Trimming down will increase computation time and should not change estimates 

   # Example of how to run regionsExtracter.sh
##./regionsExtracter.sh sample_info num_regions [out_directory] [out_name] [ref_gen]

In [35]:
angsdWrap_dir = '/data/gpfs/home/tfaske/g/src/angsd-wrapper'
regionsExt = os.path.join(angsdWrap_dir,'./regionsExtracter.sh')

In [36]:
cd $ngsF_dir

/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/ngsF


In [39]:
contigs = 10000 ## number of randomly sampled contigs
!$regionsExt $bam_list $contigs $ngsF_dir extractedRegions.txt $assembly

There are 161 BAM files listed in /data/gpfs/home/tfaske/g/jahner/bighorn/ngsF/bam_list.txt
Looking at 5466 regions...
Selecting 10000 regions:
Searching for scaffold_3716 in /data/gpfs/home/tfaske/g/jahner/bighorn/assembly/aries_genome.fa ...
Searching for scaffold_2366 in /data/gpfs/home/tfaske/g/jahner/bighorn/assembly/aries_genome.fa ...
^C


# Example extractedRegions.txt with selected chromosomes

In [40]:
extRegions_file = os.path.join(ngsF_dir,'extractedRegions.txt')

In [43]:
!head $extRegions_file

scaffold_1:
scaffold_2:
scaffold_3:
scaffold_4:
scaffold_5:
scaffold_6:
scaffold_7:
scaffold_8:
scaffold_9:
scaffold_10:


# Copy config file to ngsF dir and change settings
    confer with https://github.com/mojaveazure/angsd-wrapper/wiki/Inbreeding-Coefficient
    
    MAKE SURE N_CORES IS THE SAME AS REQUESTED IN SLURM

In [76]:
#example 
config_file = os.path.join(ngsF_dir,'Inbreeding_Coefficients_Config')

In [77]:
!cat $config_file

#!/bin/bash

set -e
set -u
set -o pipefail

#   A simple script to hold variables for the NGS_F
#   Are you using the Common_Config file?
#       If so, where is it?
COMMON=/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/ngsF

##############################################################################################
#   If we aren't using the Common_Config file, specify these variables
#   If Common_Config is specified, leave these blank
#   Define a list of samples
SAMPLE_LIST=/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/ngsF/bam_list.txt

#   Ancestral and Reference sequences
ANC_SEQ=/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/assembly/aries_genome.fa
REF_SEQ=/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/assembly/aries_genome.fa

#   Name the project
PROJECT=

#   Where do we put the outfiles?
    #   Note, the final outdirectory will be
    #   ${SCRATCH}/${PROJECT}
SCRATCH=/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/ngsF/

#   Define the region being

# Run ngsF with slurm!

    the below code with make a slurm script for you
    
    I recommend running it first through the command line and then kill it just to check if it works! 
    
    cd PATH/ngsF
    source activate angsdWrap
    angsd-wrapper Inbreeding Inbreeding_Coefficients_Config

Things you should need to change:
root_dir, angsdWrap, cpus, email, account, part 

In [92]:
root_dir = '/data/gpfs/home/tfaske/g/jahner/bighorn'
angsdWrap = '/data/gpfs/assoc/parchmanlab/tfaske/src/angsd-wrapper/./angsd-wrapper' #MAKE SURE YOU USE FULL PATH (no symbolic links)
ntasks = 2
cpus = 32
email = 'tfaske@nevada.unr.edu'
account = 'cpu-s3-sponsored-0'
part = 'cpu-s3-sponsored-0'

In [93]:
ngsF_dir = os.path.join(root_dir,'ngsF')
config_file = os.path.join(ngsF_dir,'Inbreeding_Coefficients_Config')

In [94]:
cd $ngsF_dir

/data/gpfs/assoc/parchmanlab/tfaske/jahner/bighorn/ngsF


In [97]:
def write_ngsF_sh(account,part,ntasks,cpus,email,ngsF_dir,angsdWrap,config_file):
    with open("run_ngsF.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --ntasks %d
#SBATCH --cpus-per-task %d
#SBATCH --job-name ngsF
#SBATCH --output output_ngsF.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s

## change into the ngsF directory
cd %s \n\n""" % (account,part,ntasks,cpus,email,ngsF_dir))

        #angsd-wrapper Inbreeding Inbreeding_Coefficients_Config
        o.write("""%s Inbreeding %s"""% (angsdWrap,config_file))

In [98]:
write_ngsF_sh(account,part,ntasks,cpus,email,ngsF_dir,angsdWrap,config_file)

# run in command line 
    cd PATH/ngsF
    source activate angsdWrap
    sbatch run_ngsF.sh